<a href="https://colab.research.google.com/github/dsogden/Peptide-ML/blob/main/peptide_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
ls

0.txt  1.txt  2.txt  sample_data/


In [3]:
def generate_data(input_name: str):
    file = input_name
    return np.genfromtxt(file)

a = generate_data('0.txt')
b = generate_data('1.txt')
c = generate_data('2.txt')

def array_to_dataframe(data, column_names):
    return pd.DataFrame(data, columns=column_names)

column_names = ['Frames', 'E2E', 'Phi1', 'Psi1', 'Phi2', 'Psi2', 'Phi3', 'Psi3']
df1 = array_to_dataframe(a, column_names)
df2 = array_to_dataframe(b, column_names)
df3 = array_to_dataframe(c, column_names)

df1 = df1.drop(columns='Frames', axis=1)
df2 = df2.drop(columns='Frames', axis=1)
df3 = df3.drop(columns='Frames', axis=1)

Each dataframe is composed of a column for "frames" (timestep a configuration was recorded), an end-to-end distance between the first and third amino acid's alpha carbon, and also the phi-psi dihedral angles for each of the amino acids.

In [4]:
df1

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3
0,13.135189,-55.321724,137.822891,-141.983337,128.978500,-141.861404,165.220612
1,13.027682,-64.897774,137.362518,-142.287582,133.606064,-98.886559,163.371902
2,12.901690,-47.805790,126.214409,-113.465111,129.541061,-73.677299,140.983612
3,12.494431,-69.608025,154.428467,-105.907295,127.579971,-58.139378,133.523422
4,12.442785,-80.009117,133.068466,-60.391331,132.746506,-85.038116,165.848221
...,...,...,...,...,...,...,...
4995,10.846120,-112.629898,5.526999,-54.972588,141.224411,-126.947166,133.173523
4996,12.951663,-120.800789,18.165026,-36.636044,144.836990,-156.034454,156.386108
4997,11.906729,-87.255264,-11.406045,-67.853439,139.765396,-119.077324,-162.079315
4998,11.708464,-95.077225,-14.757856,-64.236603,138.564529,-138.359085,157.903030


In [5]:
def peptide_regions(data, columns):
    n = len(columns) // 3    

    for i in range(n+1):

        Phi = data[columns[2*i]]
        Psi = data[columns[2*i+1]]

        f = []
        beta = []
        alpha_rh = []
        alpha_lh = []
        results = {}

        row = 0
        dihedral = i+1
        for phi, psi in zip(Phi, Psi):
            # f region
            if (phi >= -110 and phi <= -20) and ((psi >= -180 and psi <=-110) or (psi >= 50 and psi <=180)):
                f.append([dihedral, row, phi, psi])

            # beta region
            elif ((phi >= -180 and phi <= -110) or (phi >= 160 and phi <= 180)) and (
                (psi >= -180 and psi <=-120) or (psi >= 50 and psi <=180) or (psi >= 120 and psi <=180)):
                beta.append([dihedral, row, phi, psi])

            # alpha right-handed
            elif (phi >= -160 and phi <= -20) and (psi >= -120 and psi <=50):
                alpha_rh.append([dihedral, row, phi, psi])

            # alpha left-handed
            else:
                alpha_lh.append([dihedral, row, phi, psi])

            row += 1
    regions = ['f', 'beta', 'alpha_rh', 'alpha_lh']
    lists = [f, beta, alpha_rh, alpha_lh]
    n = len(regions)
    for i in range(n):
        results[regions[i]] = lists[i]

    return results

In [6]:
columns = ['Phi1', 'Psi1', 'Phi2', 'Psi2', 'Phi3', 'Psi3']
d1 = peptide_regions(df1, columns)
d1['alpha_rh'][0]

[3, 235, -103.63719177246094, -3.0262370109558105]

In [7]:
df1

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3
0,13.135189,-55.321724,137.822891,-141.983337,128.978500,-141.861404,165.220612
1,13.027682,-64.897774,137.362518,-142.287582,133.606064,-98.886559,163.371902
2,12.901690,-47.805790,126.214409,-113.465111,129.541061,-73.677299,140.983612
3,12.494431,-69.608025,154.428467,-105.907295,127.579971,-58.139378,133.523422
4,12.442785,-80.009117,133.068466,-60.391331,132.746506,-85.038116,165.848221
...,...,...,...,...,...,...,...
4995,10.846120,-112.629898,5.526999,-54.972588,141.224411,-126.947166,133.173523
4996,12.951663,-120.800789,18.165026,-36.636044,144.836990,-156.034454,156.386108
4997,11.906729,-87.255264,-11.406045,-67.853439,139.765396,-119.077324,-162.079315
4998,11.708464,-95.077225,-14.757856,-64.236603,138.564529,-138.359085,157.903030


In [8]:
names = ['c1', 'c2', 'c3', 'c4']
def add_columns(dataframe, n, names):
    columns = names
    for i in range(n):
        dataframe[columns] = 0
    return dataframe
n = len(names)
df1_2 = add_columns(df1, n, names)
df1_2

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,c1,c2,c3,c4
0,13.135189,-55.321724,137.822891,-141.983337,128.978500,-141.861404,165.220612,0,0,0,0
1,13.027682,-64.897774,137.362518,-142.287582,133.606064,-98.886559,163.371902,0,0,0,0
2,12.901690,-47.805790,126.214409,-113.465111,129.541061,-73.677299,140.983612,0,0,0,0
3,12.494431,-69.608025,154.428467,-105.907295,127.579971,-58.139378,133.523422,0,0,0,0
4,12.442785,-80.009117,133.068466,-60.391331,132.746506,-85.038116,165.848221,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4995,10.846120,-112.629898,5.526999,-54.972588,141.224411,-126.947166,133.173523,0,0,0,0
4996,12.951663,-120.800789,18.165026,-36.636044,144.836990,-156.034454,156.386108,0,0,0,0
4997,11.906729,-87.255264,-11.406045,-67.853439,139.765396,-119.077324,-162.079315,0,0,0,0
4998,11.708464,-95.077225,-14.757856,-64.236603,138.564529,-138.359085,157.903030,0,0,0,0


In [9]:
for key in d1.keys():
    print(key)
keys = [key for key in d1.keys()]
print(keys)
#len(d1), d1['f'][0], keys[0]

f
beta
alpha_rh
alpha_lh
['f', 'beta', 'alpha_rh', 'alpha_lh']


In [10]:
def update_dataframe(dataframe, X, names):
    ## update the additional columns with the appropriate dihedral configuration
    columns = names
    keys = [key for key in X.keys()]
    n = len(X)
    for i in range(n):
        N = len(X[keys[i]])
        for j in range(N):
            dataframe.loc[X[keys[i]][j][1], columns[i]] = i+1
    return dataframe

names = ['c1', 'c2', 'c3', 'c4']
df1_3 = update_dataframe(df1_2, d1, names)
df1_3.head(10)

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,c1,c2,c3,c4
0,13.135189,-55.321724,137.822891,-141.983337,128.978500,-141.861404,165.220612,0,2,0,0
1,13.027682,-64.897774,137.362518,-142.287582,133.606064,-98.886559,163.371902,1,0,0,0
2,12.901690,-47.805790,126.214409,-113.465111,129.541061,-73.677299,140.983612,1,0,0,0
3,12.494431,-69.608025,154.428467,-105.907295,127.579971,-58.139378,133.523422,1,0,0,0
4,12.442785,-80.009117,133.068466,-60.391331,132.746506,-85.038116,165.848221,1,0,0,0
5,13.701652,-167.813004,164.264114,-147.007904,142.194839,-57.880272,146.811218,1,0,0,0
6,12.976062,-139.881744,144.496277,-118.901993,124.804703,-58.175335,123.377052,1,0,0,0
7,13.179635,-74.760429,171.450714,-148.616943,136.147522,-98.241379,129.264496,1,0,0,0
8,12.751938,-86.455406,167.239349,-152.838730,135.184204,-99.190422,125.049675,1,0,0,0
9,12.885491,-101.286873,132.991516,-119.494202,134.786057,-84.977394,126.553001,1,0,0,0


In [11]:
df1_3[names].value_counts()

c1  c2  c3  c4
1   0   0   0     3131
0   0   3   0      985
    2   0   0      760
    0   0   4      124
dtype: int64

In [12]:
df1_3.tail(10)

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,c1,c2,c3,c4
4990,8.322098,-57.250107,-44.847828,-69.956696,132.840683,-53.923222,139.656372,1,0,0,0
4991,11.107919,-90.340858,6.431013,-89.003990,140.301010,-116.561165,178.795135,0,2,0,0
4992,12.256472,-100.260849,4.083831,-81.588623,144.066498,-153.412994,161.930542,0,2,0,0
4993,12.531058,-110.213593,17.675629,-44.608658,145.181381,-155.576965,159.667572,0,2,0,0
4994,11.552115,-102.642067,-2.650904,-50.156742,146.825699,-97.023193,148.143890,1,0,0,0
4995,10.846120,-112.629898,5.526999,-54.972588,141.224411,-126.947166,133.173523,0,2,0,0
4996,12.951663,-120.800789,18.165026,-36.636044,144.836990,-156.034454,156.386108,0,2,0,0
4997,11.906729,-87.255264,-11.406045,-67.853439,139.765396,-119.077324,-162.079315,0,2,0,0
4998,11.708464,-95.077225,-14.757856,-64.236603,138.564529,-138.359085,157.903030,0,2,0,0
4999,10.637455,-97.500061,-7.991235,-62.284107,139.190369,-118.674660,138.504471,0,2,0,0


Due to this analysis (Reference below), it appears that only one of the amino dihedral angle (tri-peptide or composed of three amino acids) configurations is contributing overall to where you would find it on a Ramachandran plot. A value of 0 indicates that the other two amino acids are in a conformation that is "disordered" and doesn't fall into the thresholds set from the above criteria. Admittedly, this is only one of the simulation sets at its very short, 10ns. Still very interesting


Mahmoud Moradi, Volodymyr Babin, Celeste Sagui, and Christopher Roland
The Journal of Physical Chemistry B 2011 115 (26), 8645-8656
DOI: 10.1021/jp203874f



In [13]:
### Update last two dataframes
columns = ['Phi1', 'Psi1', 'Phi2', 'Psi2', 'Phi3', 'Psi3']
d2 = peptide_regions(df2, columns)
d3 = peptide_regions(df3, columns)

names = ['c1', 'c2', 'c3', 'c4']
df2_2 = add_columns(df2, n, names)
df3_2 = add_columns(df3, n, names)

df2_3 = update_dataframe(df2_2, d2, names)
df3_3 = update_dataframe(df3_2, d3, names)

In [14]:
df2_3.head(5)

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,c1,c2,c3,c4
0,12.755807,-59.773178,158.493195,-152.104614,130.684219,-90.203644,169.644699,1,0,0,0
1,13.189887,-118.940315,156.270294,-158.948059,136.089386,-105.251991,136.006668,1,0,0,0
2,13.191858,-100.464157,169.845764,-153.947632,135.113312,-65.826942,146.327225,1,0,0,0
3,13.053868,-73.840317,157.636765,-160.357224,136.096558,-76.498413,176.454788,1,0,0,0
4,12.869932,-80.328773,141.339600,-142.076462,135.375595,-74.733345,156.711700,1,0,0,0


In [15]:
df1_3.tail(5)

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,c1,c2,c3,c4
4995,10.846120,-112.629898,5.526999,-54.972588,141.224411,-126.947166,133.173523,0,2,0,0
4996,12.951663,-120.800789,18.165026,-36.636044,144.836990,-156.034454,156.386108,0,2,0,0
4997,11.906729,-87.255264,-11.406045,-67.853439,139.765396,-119.077324,-162.079315,0,2,0,0
4998,11.708464,-95.077225,-14.757856,-64.236603,138.564529,-138.359085,157.903030,0,2,0,0
4999,10.637455,-97.500061,-7.991235,-62.284107,139.190369,-118.674660,138.504471,0,2,0,0


In [16]:
df2_3.tail(5)

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,c1,c2,c3,c4
4995,12.310466,-60.790138,143.039963,-71.794998,-173.966370,-96.314262,-6.954352,0,0,3,0
4996,11.946497,-80.404770,157.724655,-53.003765,-156.648636,-118.151604,27.256130,0,0,3,0
4997,11.763716,-61.928192,147.467194,-45.550194,-162.843857,-101.018097,-20.146259,0,0,3,0
4998,11.132506,-55.315868,155.194214,-54.191006,-167.754913,-146.041550,17.104572,0,0,3,0
4999,11.097328,-62.815762,149.716782,-75.121643,-150.254974,-101.713745,-16.448801,0,0,3,0


In [17]:
df3_3.head(5)

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,c1,c2,c3,c4
0,13.335229,-48.892056,132.460464,-139.626602,133.456940,-130.091125,162.159653,0,2,0,0
1,13.041007,-72.835732,135.853058,-133.061462,131.294907,-84.261635,161.089157,1,0,0,0
2,11.353611,-96.574265,119.889915,-116.853172,136.923416,-66.099030,104.197121,1,0,0,0
3,12.862085,-94.521957,160.457153,-142.650482,138.917618,-82.090408,121.215012,1,0,0,0
4,12.258504,-72.310356,160.386307,-153.531799,132.309311,-69.941345,146.490250,1,0,0,0


In [18]:
df3_3.tail(5)

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,c1,c2,c3,c4
4995,13.151813,-131.936584,104.064194,-52.900631,-171.672485,-137.436127,142.341461,0,2,0,0
4996,12.464512,-94.414726,144.345520,-60.867882,-177.277283,-101.791756,110.420837,1,0,0,0
4997,12.918749,-143.443726,173.781097,-76.849617,-170.760269,-141.233765,138.823868,0,2,0,0
4998,12.727839,-159.542084,155.168564,-30.815752,-172.883560,-129.158203,139.658920,0,2,0,0
4999,12.314855,-82.436882,167.650391,-51.033382,169.638718,-117.700195,105.023270,0,2,0,0


In [19]:
print(df1_3[names].value_counts(), '\n')
print(df2_3[names].value_counts(), '\n')
print(df3_3[names].value_counts())

c1  c2  c3  c4
1   0   0   0     3131
0   0   3   0      985
    2   0   0      760
    0   0   4      124
dtype: int64 

c1  c2  c3  c4
1   0   0   0     2329
0   0   3   0     1770
    2   0   0      861
    0   0   4       40
dtype: int64 

c1  c2  c3  c4
0   0   3   0     1859
1   0   0   0     1713
0   2   0   0     1097
    0   0   4      331
dtype: int64


For this particular tri-peptide, the f and alpha right-handed contribute the most, with replica 3 having the two split almost evenly. The simulations are performing as they should, they are all random but they are all still somewhat similar. We can proceed forward and perform our classification modeling. Although this would be a better model if I had added some bias to the systems but this data will still do the trick.

First we will try sklearn and run a couple different kinds of classifiers on it and see which ones performs the best and then we can tune it up. We need to encode the labels first and then move forward, we can also reparameterize the dihedral angles but we can leave them alone for now.

In [20]:
temp = df1_3.append(df2_3, ignore_index=True)
final = df1_3.append(temp, ignore_index=True)
final

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,c1,c2,c3,c4
0,13.135189,-55.321724,137.822891,-141.983337,128.978500,-141.861404,165.220612,0,2,0,0
1,13.027682,-64.897774,137.362518,-142.287582,133.606064,-98.886559,163.371902,1,0,0,0
2,12.901690,-47.805790,126.214409,-113.465111,129.541061,-73.677299,140.983612,1,0,0,0
3,12.494431,-69.608025,154.428467,-105.907295,127.579971,-58.139378,133.523422,1,0,0,0
4,12.442785,-80.009117,133.068466,-60.391331,132.746506,-85.038116,165.848221,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
14995,12.310466,-60.790138,143.039963,-71.794998,-173.966370,-96.314262,-6.954352,0,0,3,0
14996,11.946497,-80.404770,157.724655,-53.003765,-156.648636,-118.151604,27.256130,0,0,3,0
14997,11.763716,-61.928192,147.467194,-45.550194,-162.843857,-101.018097,-20.146259,0,0,3,0
14998,11.132506,-55.315868,155.194214,-54.191006,-167.754913,-146.041550,17.104572,0,0,3,0


In [21]:
final['Label'] = final['c1'] + final['c2'] + final['c3'] + final['c4']
final

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,c1,c2,c3,c4,Label
0,13.135189,-55.321724,137.822891,-141.983337,128.978500,-141.861404,165.220612,0,2,0,0,2
1,13.027682,-64.897774,137.362518,-142.287582,133.606064,-98.886559,163.371902,1,0,0,0,1
2,12.901690,-47.805790,126.214409,-113.465111,129.541061,-73.677299,140.983612,1,0,0,0,1
3,12.494431,-69.608025,154.428467,-105.907295,127.579971,-58.139378,133.523422,1,0,0,0,1
4,12.442785,-80.009117,133.068466,-60.391331,132.746506,-85.038116,165.848221,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14995,12.310466,-60.790138,143.039963,-71.794998,-173.966370,-96.314262,-6.954352,0,0,3,0,3
14996,11.946497,-80.404770,157.724655,-53.003765,-156.648636,-118.151604,27.256130,0,0,3,0,3
14997,11.763716,-61.928192,147.467194,-45.550194,-162.843857,-101.018097,-20.146259,0,0,3,0,3
14998,11.132506,-55.315868,155.194214,-54.191006,-167.754913,-146.041550,17.104572,0,0,3,0,3


In [22]:
final = final.drop(columns=['c1', 'c2', 'c3', 'c4'], axis=1)

In [23]:
final

,E2E,Phi1,Psi1,Phi2,Psi2,Phi3,Psi3,Label
0,13.135189,-55.321724,137.822891,-141.983337,128.978500,-141.861404,165.220612,2
1,13.027682,-64.897774,137.362518,-142.287582,133.606064,-98.886559,163.371902,1
2,12.901690,-47.805790,126.214409,-113.465111,129.541061,-73.677299,140.983612,1
3,12.494431,-69.608025,154.428467,-105.907295,127.579971,-58.139378,133.523422,1
4,12.442785,-80.009117,133.068466,-60.391331,132.746506,-85.038116,165.848221,1
...,...,...,...,...,...,...,...,...
14995,12.310466,-60.790138,143.039963,-71.794998,-173.966370,-96.314262,-6.954352,3
14996,11.946497,-80.404770,157.724655,-53.003765,-156.648636,-118.151604,27.256130,3
14997,11.763716,-61.928192,147.467194,-45.550194,-162.843857,-101.018097,-20.146259,3
14998,11.132506,-55.315868,155.194214,-54.191006,-167.754913,-146.041550,17.104572,3


In [24]:
from sklearn.model_selection import train_test_split

In [25]:
final.columns

Index(['E2E', 'Phi1', 'Psi1', 'Phi2', 'Psi2', 'Phi3', 'Psi3', 'Label'], dtype='object')

In [26]:
features = ['E2E', 'Phi1', 'Psi1', 'Phi2', 'Psi2', 'Phi3', 'Psi3']
y = final['Label']
X = final[features]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [27]:
labels = ['f', 'beta', 'alpha_rh', 'alpha_lh']
len(X_train), len(X_test)

(12000, 3000)

1. Support Vector Machine

In [28]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

svc = SVC(gamma="auto", random_state=42)
svc.fit(X_train, y_train)



SVC(gamma='auto', random_state=42)

In [29]:
svc.classes_

array([1, 2, 3, 4])

In [30]:
score = cross_val_score(svc, X_train, y_train, cv=10, scoring="accuracy")
print(score.mean())

0.7557499999999999


In [31]:
predictions = svc.predict(X_test)

In [32]:
test_values = y_test.to_numpy()

In [33]:
predictions[1000], test_values[1000]

(1, 3)

2. Random Forest

In [34]:
from sklearn.ensemble import RandomForestClassifier

forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
forest_clf.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [35]:
forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

0.99925

99% accuracy is a little questionable for training. 

In [36]:
predictions = forest_clf.predict(X_test)

In [37]:
predictions

array([1, 2, 3, ..., 2, 1, 2])

In [38]:
def calculate_error(x, y):
    n = len(x)
    counter = 0
    for i in range(n):
        if x[i] != y[i]:
            counter += 1
    return counter / n * 100

percent_error = calculate_error(predictions, test_values)
print(f'Percent Error from RFC: {percent_error}%')


Percent Error from RFC: 0.06666666666666667%


In [54]:
print(f"{final['Label'].value_counts() / len(final)}\n")
print(f"{y_test.value_counts() / len(y_test)}\n")
print(f"{y_train.value_counts() / len(y_train)}")

1    0.572733
3    0.249333
2    0.158733
4    0.019200
Name: Label, dtype: float64

1    0.574667
3    0.244000
2    0.162667
4    0.018667
Name: Label, dtype: float64

1    0.572250
3    0.250667
2    0.157750
4    0.019333
Name: Label, dtype: float64


Try with less estimators

In [55]:
forest_clf = RandomForestClassifier(n_estimators=10, random_state=42)
forest_clf.fit(X_train, y_train)

forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

0.9987499999999999

Try with 500 estimators

In [56]:
forest_clf = RandomForestClassifier(n_estimators=500, random_state=42)
forest_clf.fit(X_train, y_train)

forest_scores = cross_val_score(forest_clf, X_train, y_train, cv=10)
forest_scores.mean()

0.9992500000000002

100 estimators is plenty and is verified through the CV. 